In [1]:
import numpy as np

In [7]:
import matplotlib.pyplot as plt
import os
import pandas as pd


In [3]:
def load_all_npy_files(directory):
    """
    Load all .npy files from a specified directory.

    Parameters:
    - directory (str): Path to the directory containing .npy files.

    Returns:
    - A dictionary with filenames as keys and loaded numpy arrays as values.
    """
    files = [f for f in os.listdir(directory) if f.endswith('.npy')]
    data = {}

    for f in files:
        full_path = os.path.join(directory, f)
        data[f] = np.load(full_path)

    return data


rs_list = ['kl', 'kl-iw', 'multinomial', 'systematic', 'stratified']

all_data = {}
for rs in rs_list:
    directory_path = './results/{}/'.format(rs)
    all_data[rs] = load_all_npy_files(directory_path)




In [19]:
data = all_data

# Extract all unique primary keys from the original dictionary.
primary_keys = set(key_inner.split("_")[0] for val_outer in data.values() for key_inner in val_outer.keys())

# Create a DataFrame for each unique primary key.
dfs = {primary_key: pd.DataFrame() for primary_key in primary_keys}

# Populate DataFrames
# Populate DataFrames
for outer_key, val_outer in data.items():
    for key_inner, val_inner in val_outer.items():
        key_parts = key_inner.replace(".npy", "").split("_")

        # Get the name and associated value pairs.
        col_values = {key_parts[i]: key_parts[i + 1] for i in range(1, len(key_parts), 2)}
        # Add the outer key as a column with the actual value from the dictionary.
        col_values[outer_key] = val_inner.item()

        # Append data to the corresponding DataFrame.
        primary_key = key_parts[0]
        new_row = pd.DataFrame([col_values])
        dfs[primary_key] = pd.concat([dfs[primary_key], new_row], ignore_index=True)

# # Convert DataFrames to LaTeX tables.
# for primary_key, df in dfs.items():
#     print(f"LaTeX table for {primary_key}:")
#     # Replace NaN values with "-"
#     df_filled = df.fillna("-")
#     # Convert to LaTeX without index and print
#     print(df_filled.style.to_latex())
#     print("\n")

# Columns to compare for lowest values.
compare_cols = ["kl", "kl-iw", "multinomial", "systematic", "stratified"]

# Convert DataFrames to LaTeX tables.
for primary_key, df in dfs.items():
    print(f"LaTeX table for {primary_key}:")

    # Replace NaN values with "-"
    df_filled = df.fillna("-")

    # Find the global minimum across specified columns.
    min_val = df_filled[compare_cols].apply(pd.to_numeric, errors='coerce').min().min()

    # Define a function to bold the lowest values among specified columns.
    def highlight_min(s):
        # Return appropriate style strings
        return ['font-weight: bold' if v == min_val else '' for v in s]

    # Apply the styling and convert to LaTeX.
    styled = df_filled.style.apply(highlight_min, subset=compare_cols, axis=1)
    print(styled.to_latex())
    print("\n")


LaTeX table for avgI-0:
\begin{tabular}{llllllllllll}
 & dimensions & N & continuousT & dt & nruns & truthparticles & kl & kl-iw & multinomial & systematic & stratified \\
0 & 10 & 100 & 6 & 0.01 & 50 & 80000 & 20.097425 & - & - & - & - \\
1 & 10 & 100 & 6 & 0.01 & 50 & 80000 & - & \font-weightbold 19.920384 & - & - & - \\
2 & 10 & 100 & 6 & 0.01 & 50 & 80000 & - & - & 20.997391 & - & - \\
3 & 10 & 100 & 6 & 0.01 & 50 & 80000 & - & - & - & 19.939955 & - \\
4 & 10 & 100 & 6 & 0.01 & 50 & 80000 & - & - & - & - & 19.954173 \\
\end{tabular}



LaTeX table for varianceloglik:
\begin{tabular}{llllllllllll}
 & dimensions & N & continuousT & dt & nruns & truthparticles & kl & kl-iw & multinomial & systematic & stratified \\
0 & 10 & 100 & 6 & 0.01 & 50 & 80000 & \font-weightbold 5641968.115969 & - & - & - & - \\
1 & 10 & 100 & 6 & 0.01 & 50 & 80000 & - & 7096163.259868 & - & - & - \\
2 & 10 & 100 & 6 & 0.01 & 50 & 80000 & - & - & 27481041.480697 & - & - \\
3 & 10 & 100 & 6 & 0.01 & 50 & 80000 

In [21]:
# # Open a single .tex file to save all the tables.
# with open("./results/all_tables_new.tex", "w") as file:
#
#     # Convert DataFrames to LaTeX tables and save them to the .tex file.
#     for primary_key, df in dfs.items():
#
#         # Write a section header for each table.
#         file.write(f"\\section*{{Table for {primary_key}}}\n")
#
#         # Select only the desired columns.
#         df_selected = df[compare_cols].copy()
#
#         # Replace NaN values with "-"
#         df_filled = df_selected.fillna("-")
#
#         # Find the global minimum across specified columns.
#         min_val = df_filled.apply(pd.to_numeric, errors='coerce').min().min()
#
#         # Define a function to bold the lowest values among specified columns.
#         def highlight_min(s):
#             # Return appropriate style strings
#             return ['font-weight: bold' if v == min_val else '' for v in s]
#
#         # Apply the styling and convert to LaTeX.
#         styled = df_filled.style.apply(highlight_min, subset=compare_cols, axis=1)
#
#         # Write to the .tex file.
#         file.write(styled.to_latex())
#         file.write("\n\n")


In [23]:
dfs = {}
for algorithm_type, metrics in data.items():
    for metric, value in metrics.items():
        metric_name = metric.split("_")[0]  # Extracting the metric name, e.g., 'stdI-0'

        # If this metric DataFrame hasn't been created yet, create it.
        if metric_name not in dfs:
            dfs[metric_name] = pd.DataFrame()

        # Add the value to the DataFrame.
        dfs[metric_name][algorithm_type] = [float(value)]

# Now, you have `dfs` dict with each DataFrame ready to be converted to LaTeX as before
compare_cols = ["kl", "kl-iw", "multinomial", "systematic", "stratified"]

with open("./results/all_tables_simple.tex", "w") as file:

    # Convert DataFrames to LaTeX tables and save them to the .tex file.
    for primary_key, df in dfs.items():

        # Write a section header for each table.
        file.write(f"\\section*{{Table for {primary_key}}}\n")

        # Find the global minimum across specified columns.
        min_val = df.min().min()

        # Define a function to bold the lowest values among specified columns.
        def highlight_min(s):
            return ['font-weight: bold' if v == min_val else '' for v in s]

        # Apply the styling and convert to LaTeX.
        styled = df.style.apply(highlight_min, subset=compare_cols, axis=1)

        # Write to the .tex file.
        file.write(styled.to_latex())
        file.write("\n\n")
